In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
import glob
from google.cloud import bigquery
import re

In [2]:
client = bigquery.Client()

In [3]:
query = '''SELECT
  cbo_2002,
  CASE
    WHEN natureza_juridica IN ('1015','1040', '1074', '1104','1139', '1163', '1252', '1287', '1317', '1341') THEN 'Federal'
    WHEN natureza_juridica IN ('1023','1058', '1082', '1112', '1147', '1171', '1236','1260',  '1295', '1325') THEN 'Estadual'
    WHEN natureza_juridica IN ('1031','1066', '1120', '1155', '1180', '1244', '1279', '1309', '1333') THEN 'Municipal'
    ELSE 'Outros'
  END AS esfera,
  CASE
    WHEN sexo = '1' THEN 'Masculino'
    WHEN sexo = '2' THEN 'Feminino'
    ELSE 'Não identificado'
  END AS sexo,
 valor_remuneracao_media,
FROM `basedosdados.br_me_rais.microdados_vinculos`
WHERE (natureza_juridica LIKE "1%"
  OR natureza_juridica IN ('2011', '2038'))
  AND natureza_juridica != '1228'
  AND cbo_2002 NOT LIKE "0%" -- retirando os militares.
  AND vinculo_ativo_3112 = '1'
  AND ano = 2022'''

In [5]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df # usa essa

Downloading: 100%|██████████|


,cbo_2002,esfera,sexo,valor_remuneracao_media
0,911305,Outros,Feminino,622.83
1,422105,Municipal,Feminino,1408.18
2,412110,Municipal,Feminino,1212.00
3,341230,Municipal,Masculino,1288.08
4,239410,Municipal,Feminino,1212.00
...,...,...,...,...
9966399,782305,Outros,Masculino,4036.67
9966400,782305,Outros,Masculino,4196.73
9966401,782305,Municipal,Masculino,4619.22
9966402,782305,Municipal,Masculino,4038.66


In [6]:
df = df.rename(columns={'cbo_2002':'codigo',
                        'sexo':'genero'})


In [7]:
df['codigo'] = df['codigo'].astype(int)

In [8]:
df.dtypes

codigo                       int32
esfera                      object
genero                      object
valor_remuneracao_media    float64
dtype: object

In [9]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CBO\\ESTRUTURA CBO')

In [10]:
os.listdir()

['CBO2002 - Sinonimo.csv',
 'CBO2002 - PerfilOcupacional.csv',
 'CBO2002 - Ocupacao.csv',
 'CBO2002 - SubGrupo Principal.csv',
 'CBO2002 - Grande Grupo.csv',
 'CBO2002 - Familia.csv',
 'CBO2002 - SubGrupo.csv',
 'titulo_funcoes_cbo.xlsx',
 'CBO2002 - Ocupacao.gsheet',
 'CBO2002 - SubGrupo Principal.gsheet',
 'CBO2002 - SubGrupo.gsheet',
 'CBO2002 - Ocupação.xlsx',
 'comparando dados CBO.gsheet',
 'CBO2002 - Grande Grupo.gsheet']

In [11]:
cbo = pd.read_excel("CBO2002 - Ocupação.xlsx")
cbo

,CODIGO,TITULO,Agrupamento
0,10105,Oficial general da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
1,10110,Oficial general do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
2,10115,Oficial general da marinha,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
3,10205,Oficial da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
4,10210,Oficial do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2709,992210,Encarregado de equipe de conservação de vias p...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2710,992215,Operador de ceifadeira na conservação de vias ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2711,992220,Pedreiro de conservação de vias permanentes (e...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...


In [12]:
cbo = cbo.rename(columns=
                 {
                     'CODIGO':'codigo',
                     'TITULO':'nome_ocupacao',
                     'Agrupamento':'grupo_ocupacao'
                 })

In [13]:
cbo_merge = df.merge(cbo, how='left', on='codigo')
cbo_merge 

,codigo,esfera,genero,valor_remuneracao_media,nome_ocupacao,grupo_ocupacao
0,911305,Outros,Feminino,622.83,"Mecânico de manutenção de máquinas, em geral",TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
1,422105,Municipal,Feminino,1408.18,"Recepcionista, em geral",TRABALHADORES DE SERVIÇOS ADMINISTRATIVOS
2,412110,Municipal,Feminino,1212.00,Digitador,TRABALHADORES DE SERVIÇOS ADMINISTRATIVOS
3,341230,Municipal,Masculino,1288.08,Piloto fluvial,TÉCNICOS DE NIVEL MÉDIO
4,239410,Municipal,Feminino,1212.00,Orientador educacional,PROFISSIONAIS DE ENSINO
...,...,...,...,...,...,...
9966399,782305,Municipal,Masculino,2413.21,Motorista de carro de passeio,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...
9966400,782305,Municipal,Masculino,4641.35,Motorista de carro de passeio,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...
9966401,782305,Municipal,Masculino,4579.12,Motorista de carro de passeio,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...
9966402,782305,Municipal,Masculino,4934.93,Motorista de carro de passeio,TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS I...


In [14]:
cod_obsoleto = cbo_merge[cbo_merge['nome_ocupacao'].isna()] #codigos excluidos ou atualizados para outro
cod_obsoleto #5702 rows

,codigo,esfera,genero,valor_remuneracao_media,nome_ocupacao,grupo_ocupacao
1324,999999,Outros,Feminino,0.00,NaN,NaN
3396,999999,Municipal,Masculino,1812.00,NaN,NaN
4232,517335,Outros,Masculino,11958.63,NaN,NaN
5797,517335,Outros,Masculino,16125.48,NaN,NaN
7154,517335,Outros,Masculino,6383.21,NaN,NaN
...,...,...,...,...,...,...
9535441,999999,Outros,Masculino,0.00,NaN,NaN
9535468,999999,Estadual,Masculino,447.07,NaN,NaN
9540038,999999,Outros,Masculino,3574.69,NaN,NaN
9540328,517335,Outros,Masculino,8247.10,NaN,NaN


In [15]:
cod_obsoleto['codigo'].unique()

array([999999, 517335, 373205, 373210, 373115, 373215, 322105, 322120,
       322125, 722225, 322115])

In [16]:
adicionar_nome = {
    999999: {'nome_ocupacao': 'Desconhecido Ou Não Informado', 'grupo_ocupacao': 'Trabalhadores Em Serviços De Reparação E Manutenção'},
    373205: {'nome_ocupacao': 'Técnico De Sistemas Audiovisuais', 'grupo_ocupacao': 'Técnicos De Nivel Médio'}, 
    517335: {'nome_ocupacao': 'Guarda Portuário', 'grupo_ocupacao': 'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados'},
    322105: {'nome_ocupacao': 'Técnico Em Acupuntura', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    322120: {'nome_ocupacao': 'Massoterapeuta', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    373115: {'nome_ocupacao': 'Operador De Externa (Rádio)', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    373210: {'nome_ocupacao': 'Operador De Controle Mestre', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    322125: {'nome_ocupacao': 'Terapeuta Holístico', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    373215: {'nome_ocupacao': 'Técnico Em Operação De Equipamentos De Transmissão/Recepção De Televisão', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    322115: {'nome_ocupacao': 'Técnico Em Quiropraxia', 'grupo_ocupacao': 'Técnicos De Nivel Médio'},
    722225: {'nome_ocupacao': 'Operador De Máquina De Fundir Sob Pressão', 'grupo_ocupacao': 'Trabalhadores Da Produção De Bens E Serviços Industriais'}
}


In [17]:
def adicionar(row):
    if pd.isna(row['nome_ocupacao']) and row['codigo'] in adicionar_nome:
        return pd.Series({
            'nome_ocupacao': adicionar_nome[row['codigo']]['nome_ocupacao'],
            'grupo_ocupacao': adicionar_nome[row['codigo']]['grupo_ocupacao']
        })
    return pd.Series({'nome_ocupacao': row['nome_ocupacao'], 'grupo_ocupacao': row['grupo_ocupacao']})

In [18]:
cbo_merge[['nome_ocupacao', 'grupo_ocupacao']] = cbo_merge.apply(adicionar, axis=1)

In [19]:
cbo_merge[cbo_merge['nome_ocupacao']=='Técnico de sistemas audiovisuais'] # 607

,codigo,esfera,genero,valor_remuneracao_media,nome_ocupacao,grupo_ocupacao
5679,373130,Federal,Masculino,6560.08,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
16454,373130,Federal,Feminino,4334.44,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
19581,373130,Federal,Masculino,4320.12,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
23486,373130,Federal,Masculino,4304.91,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
29938,373130,Federal,Masculino,3944.45,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
...,...,...,...,...,...,...
9528685,373130,Federal,Masculino,4710.49,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
9528686,373130,Federal,Masculino,4370.66,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
9528687,373130,Federal,Masculino,3860.43,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO
9528969,373130,Federal,Feminino,5112.01,Técnico de sistemas audiovisuais,TÉCNICOS DE NIVEL MÉDIO


In [20]:
atualizar_codigo = {
    373205: 373130,
    517335: 517235,
    322105: 322705,
    322120: 322715,
    373210: 373135,
    322125: 322720,
    322115: 322710
}

In [21]:
cbo_merge['codigo'] = cbo_merge['codigo'].replace(atualizar_codigo)


In [22]:
cbo_merge['nome_ocupacao'] = cbo_merge['nome_ocupacao'].str.title()


In [23]:
cbo_merge['grupo_ocupacao'] = cbo_merge['grupo_ocupacao'].str.title()


In [24]:
cbo_merge[cbo_merge['nome_ocupacao']=='Guarda Portuário'] #Tudo padronizado

,codigo,esfera,genero,valor_remuneracao_media,nome_ocupacao,grupo_ocupacao
4232,517235,Outros,Masculino,11958.63,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
5797,517235,Outros,Masculino,16125.48,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
7154,517235,Outros,Masculino,6383.21,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
7322,517235,Outros,Masculino,8263.66,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
7323,517235,Outros,Masculino,12526.78,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
...,...,...,...,...,...,...
9535148,517235,Outros,Masculino,16350.07,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
9536248,517235,Outros,Masculino,11347.17,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
9537787,517235,Municipal,Masculino,9706.94,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."
9540328,517235,Outros,Masculino,8247.10,Guarda Portuário,"Trabalhadores Dos Serviços, Vendedores Do Comé..."


In [25]:
cbo_merge.columns

Index(['codigo', 'esfera', 'genero', 'valor_remuneracao_media',
       'nome_ocupacao', 'grupo_ocupacao'],
      dtype='object')

In [25]:
cbo_merge

,codigo,esfera,genero,valor_remuneracao_media,nome_ocupacao,grupo_ocupacao
0,142405,Outros,Feminino,0.00,Gerente De Compras,"Membros Superiores Do Poder Público, Dirigente..."
1,411050,Outros,Feminino,0.00,Agente De Microcrédito,Trabalhadores De Serviços Administrativos
2,411050,Outros,Masculino,0.00,Agente De Microcrédito,Trabalhadores De Serviços Administrativos
3,999999,Outros,Masculino,0.00,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...
4,715505,Municipal,Masculino,0.00,Carpinteiro,Trabalhadores Da Produção De Bens E Serviços I...
...,...,...,...,...,...,...
9966399,782305,Estadual,Masculino,4301.88,Motorista De Carro De Passeio,Trabalhadores Da Produção De Bens E Serviços I...
9966400,782305,Municipal,Masculino,5719.94,Motorista De Carro De Passeio,Trabalhadores Da Produção De Bens E Serviços I...
9966401,782305,Municipal,Masculino,6591.38,Motorista De Carro De Passeio,Trabalhadores Da Produção De Bens E Serviços I...
9966402,782305,Outros,Masculino,4968.52,Motorista De Carro De Passeio,Trabalhadores Da Produção De Bens E Serviços I...


In [26]:
cbo_merge['grupo_ocupacao'].unique()

array(['Trabalhadores Em Serviços De Reparação E Manutenção',
       'Trabalhadores De Serviços Administrativos',
       'Técnicos De Nivel Médio', 'Profissionais De Ensino',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Ciência De Ensino Superior',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Profissionais De Medicina, Enfermagem E Afins',
       'Profissionais De Artes De Ensino Superior'], dtype=object)

In [27]:
#incluir apenas o nome e subir pro datalake
#agregar apenas no final
## ADICIONAR O N
#
cbo_merge_agrupado = cbo_merge.groupby(['codigo', 'esfera', 'genero', 'nome_ocupacao', 'grupo_ocupacao']).agg(
                                                                                                     remuneracao_media=('valor_remuneracao_media', 'mean'), 
                                                                                                     remuneracao_mediana=('valor_remuneracao_media', 'median'),
                                                                                                     total=('valor_remuneracao_media','count')).reset_index()
cbo_merge_agrupado

,codigo,esfera,genero,nome_ocupacao,grupo_ocupacao,remuneracao_media,remuneracao_mediana,total
0,111215,Federal,Feminino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",27476.110000,27476.110,1
1,111215,Federal,Masculino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",23741.399231,27474.850,13
2,111215,Municipal,Feminino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",6420.296667,6701.200,3
3,111215,Municipal,Masculino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",5835.045000,5880.555,6
4,111220,Estadual,Feminino,Secretário - Executivo,"Membros Superiores Do Poder Público, Dirigente...",5546.056349,5068.845,304
...,...,...,...,...,...,...,...,...
8884,999999,Federal,Masculino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,20653.659660,24500.440,147
8885,999999,Municipal,Feminino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,4318.379723,4035.950,469
8886,999999,Municipal,Masculino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,4776.027899,3902.610,119
8887,999999,Outros,Feminino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,599.227568,0.000,1147


In [28]:
cbo_merge_agrupado['remuneracao_media'] = cbo_merge_agrupado['remuneracao_media'].round(2)


In [29]:
cbo_merge_agrupado['remuneracao_mediana'] = cbo_merge_agrupado['remuneracao_mediana'].round(2)


In [30]:
cbo_merge_agrupado

,codigo,esfera,genero,nome_ocupacao,grupo_ocupacao,remuneracao_media,remuneracao_mediana,total
0,111215,Federal,Feminino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",27476.11,27476.11,1
1,111215,Federal,Masculino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",23741.40,27474.85,13
2,111215,Municipal,Feminino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",6420.30,6701.20,3
3,111215,Municipal,Masculino,Ministro De Estado,"Membros Superiores Do Poder Público, Dirigente...",5835.05,5880.56,6
4,111220,Estadual,Feminino,Secretário - Executivo,"Membros Superiores Do Poder Público, Dirigente...",5546.06,5068.84,304
...,...,...,...,...,...,...,...,...
8884,999999,Federal,Masculino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,20653.66,24500.44,147
8885,999999,Municipal,Feminino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,4318.38,4035.95,469
8886,999999,Municipal,Masculino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,4776.03,3902.61,119
8887,999999,Outros,Feminino,Desconhecido Ou Não Informado,Trabalhadores Em Serviços De Reparação E Manut...,599.23,0.00,1147


In [31]:
cbo_merge_agrupado['total'].unique()

array([   1,   13,    3, ...,  469, 1147, 2593], dtype=int64)

In [32]:
cbo_merge_agrupado['ano'] = 2022

In [33]:
cbo_merge_agrupado = cbo_merge_agrupado[['ano','codigo','nome_ocupacao','grupo_ocupacao','esfera','genero','remuneracao_media','remuneracao_mediana', 'total']]

In [34]:
cbo_merge_agrupado.dtypes

ano                      int64
codigo                   int64
nome_ocupacao           object
grupo_ocupacao          object
esfera                  object
genero                  object
remuneracao_media      float64
remuneracao_mediana    float64
total                    int64
dtype: object

In [36]:
cbo_merge_agrupado['grupo_ocupacao'].unique()

array(['Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Medicina, Enfermagem E Afins',
       'Profissionais De Ciência De Ensino Superior',
       'Profissionais De Ensino',
       'Profissionais De Artes De Ensino Superior',
       'Técnicos De Nivel Médio',
       'Trabalhadores De Serviços Administrativos',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores Em Serviços De Reparação E Manutenção'],
      dtype=object)

In [43]:
def transformar(nome):
    nome = re.sub(r"\bNivel\b", "Nível", nome)
    return nome

In [44]:
cbo_merge_agrupado['grupo_ocupacao'] = cbo_merge_agrupado['grupo_ocupacao'].apply(transformar)

In [45]:
cbo_merge_agrupado['grupo_ocupacao'].unique()

array(['Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Medicina, Enfermagem E Afins',
       'Profissionais De Ciência De Ensino Superior',
       'Profissionais De Ensino',
       'Profissionais De Artes De Ensino Superior',
       'Técnicos De Nível Médio',
       'Trabalhadores De Serviços Administrativos',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores Em Serviços De Reparação E Manutenção'],
      dtype=object)

In [35]:
dataset_ref = client.dataset('perfil_remuneracao')

In [47]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('codigo', 'INTEGER', description= 'Código da Classificação Brasileira de Ocupações (CBO)'),
          bigquery.SchemaField('nome_ocupacao', 'STRING', description= 'Nome da ocupação na CBO'),
          bigquery.SchemaField('grupo_ocupacao', 'STRING', description= 'Agrupamento da ocupação'),
          bigquery.SchemaField('esfera', 'STRING', description= 'Nível da esfera do governo referente da observação'),                    
          bigquery.SchemaField('genero', 'STRING', description= 'Gênero autodeclarado ou não'),                    
          bigquery.SchemaField('remuneracao_media', 'FLOAT', description= 'Remuneração media dos vinculos observados'),
          bigquery.SchemaField('remuneracao_mediana', 'FLOAT', description= 'Remuneração mediana dos vinculos observados'),
          bigquery.SchemaField('total','INTEGER', description='Total agrupado')
]

In [48]:
table_ref = dataset_ref.table('RAIS_cbo_media_mediana')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(cbo_merge_agrupado, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=ac46080f-fda9-416c-876c-0c6ac94cde5d>